In [1]:
from langchain_groq import ChatGroq
from langchain_openai import ChatOpenAI,OpenAIEmbeddings #this to above classes we used to interact with llm model.
from dotenv import load_dotenv
from langgraph.graph import StateGraph,END,START #this class we used to build stateful workflows in graphical form mei.
from pydantic import BaseModel,Field,computed_field
from typing import Annotated,List,Dict,TypedDict,Optional
from langchain_core.output_parsers import PydanticOutputParser,StrOutputParser
from langchain_core.prompts import PromptTemplate
from langchain_huggingface import ChatHuggingFace,HuggingFaceEndpoint

load_dotenv()

True

# step:1) creating a model object

In [2]:
#Groq Model object
model1 = ChatGroq(
    model="groq/compound-mini",
    temperature=0.2
)
model1

#openAI model object.
model2 = ChatOpenAI(temperature=0.2)

## Hugging Face endpoint define karo OPen source Model chat or Generation Model.
llm = HuggingFaceEndpoint(
    repo_id="mistralai/Mistral-7B-Instruct-v0.3",
    task="text-generation",
    max_new_tokens=512,
    do_sample=False,
)

# Chat model object banao
model3 = ChatHuggingFace(llm=llm)

# using pydantic class we are defining the structure field

In [3]:
from typing import Literal
class SentimentSchema(BaseModel):
    review : Annotated[str,Field(...,description="The user-provided review text.")]
    sentiment :Annotated[
        Optional[
            Literal['Positive','Negative','Unknown']
            ],
        Field(None,
              description="The sentiment identified from the review. Must be 'Positive', 'Negative', or 'Unknown'.")]

In [4]:
class DiagnosisSchema(BaseModel):
    Issue: Annotated[
        str,
        Field(..., description="Identify the main issue reported by the customer.")
    ]

    Urgency: Annotated[
        Literal["High", "Medium", "Low"],
        Field(..., description="Determine the urgency level of the issue.")
    ]

    Tone: Annotated[
        Literal["frustrated", "disappointed", "angry", "neutral"],
        Field(..., description="Describe the emotional tone of the review.")
    ]

# different type of parser object forming

In [5]:
# creating an pydantic output parser class object
parser = StrOutputParser()

pydantic_parser = PydanticOutputParser(pydantic_object=SentimentSchema)
pydantic_parser

PydanticOutputParser(pydantic_object=<class '__main__.SentimentSchema'>)

In [6]:
#creating pydantic parser object for dignonis schema class
dignosis_parser = PydanticOutputParser(pydantic_object=DiagnosisSchema)
dignosis_parser

PydanticOutputParser(pydantic_object=<class '__main__.DiagnosisSchema'>)

# step:1) define the state or memory class with help of Typedict futhure we can create a graph object from stategraph class

In [7]:
class ReviewState(TypedDict):
    review : str
    sentiment: Literal['Positive','Negative','Unknown']
    diagnosis : dict #why dict bcoz multiple factor we are dignosing
    response : str #this will handle pos or neg response based on user query

In [8]:
#creating a graph object by using stategraph class
graph = StateGraph(ReviewState)
graph

# step:2) adding nodes and edges to the graph

In [9]:
#defining the sentiment_analysis nodes.
def sentiment_analysis(state:ReviewState) -> ReviewState:
    #now fetching the review text from state class.
    text = state['review']
    
    #now creating a structure prompt that fetching the sentiment from review.
    prompt = PromptTemplate(
        template="""
        
        You are a sentiment analysis expert.
        Read the following user review carefully and classify its sentiment.

        Review:
        {review}

        Your task:
        - Identify whether the sentiment is **Positive**, **Negative**, or **Unknown**.
        - Do not hallucinate; if the sentiment is unclear, return **Unknown**.

        Return the result in the following format:
        {format_instructions}
        """,
            
        input_variables=['review'],
        partial_variables={'format_instructions':pydantic_parser.get_format_instructions()}
        
    )
    
    #now passing the prompt to llm model to get sentiment structure creating sequential chain.
    chain = prompt | model1 | pydantic_parser
    
    #now invoking the chain.
    result = chain.invoke({'review':text})
    
    #now updating the memory with sentiment and returning the partial state .
    
    return {
        'sentiment':result.sentiment
    }
    

In [10]:
def run_dignosis(state:ReviewState) ->ReviewState:
    #fetching the sentiment or review from state memory class
    sentiment = state['sentiment']
    review = state['review']
    
    #creating a prompt structure instruction based on negative sentiment or response
    prompt_diagnosis = PromptTemplate(
        template="""
        You are a customer support assistant.
        The following is a **negative review** from a customer:

        Review: "{review}"

        Return the result in this structured format:
        {format_instructions}
        """,
        input_variables=["review"],
        partial_variables={'format_instructions':dignosis_parser.get_format_instructions()}
        
    )
    #now passing the prompt to llm model to get sentiment structure creating sequential chain.
    chain = prompt_diagnosis | model1 | dignosis_parser
    
    #now invoking the chain.
    result = chain.invoke({'review':review})
    
    #now updating the memory with sentiment and returning the partial state .
    return {
        'diagnosis':result.model_dump() # convert pydantic object into dict
    }

In [11]:
def positive_response(state:ReviewState) ->ReviewState:
    #fetching the sentiment from state class
    sent = state['sentiment']
    
    # Create a structured prompt for positive sentiment
    prompt_positive = PromptTemplate(
        template="""
        The customer left a positive review. 
        Review: "{review}"

        Task:
        Write a polite thank-you response to the customer 
        in 1–2 short sentences, expressing appreciation
        and also ask user to give feedback on our website.
        """,
        input_variables=["review"]
    )
    
    #passing the prompt to LLM model show forming sequential chain.
    chain = prompt_positive | model1 | parser
    
    #invoking the chain.
    result = chain.invoke({'review':sent})
    
    #updating the response in state or memeory class and return partial state.
    return {
        'response':result
    }
    
    

In [12]:
def negative_response(state:ReviewState) ->ReviewState:
    #fetching the dignosis result or negative review from state class.
    dig = state['diagnosis']
    revie = state['review']
    
    # create a structured empathetic response prompt
    prompt_negative = PromptTemplate(
        template="""
        You are a helpful customer support assistant. 
        A customer has left a negative review about an electronic device. 
        You have the review text and a diagnosis of the problem.

        Review:
        {review}

        Diagnosis:
        - Issue: {Issue}
        - Urgency: {Urgency}
        - Tone: {Tone}

        Task:
        Write a professional and empathetic response to the customer in 1–2 short sentences. 
        Acknowledge their issue clearly, show understanding of their feelings, 
        and politely suggest possible next steps or support.

        Response:
        """,
        input_variables=["review", "Issue", "Urgency", "Tone"]
    )
    # build a chain → prompt → model
    chain = prompt_negative | model1 | parser # model2 is your LLM

    # run the chain with the review + unpacked diagnosis dict
    response = chain.invoke({
        "review": revie,
        **dig  # expands dict into Issue, Urgency, Tone
    })

    # update state with generated reply
    return {"response": response}
    
    

# defining helper function that route the condition to nodes

In [13]:
def route_condition(state:ReviewState) ->Literal['run_dignosis','positive_response']:
    #fetching the sentiment from state class
    sent = state['sentiment']
    
    if sent == "Positive":
        return "positive_response"
    elif sent == "Negative":
        return "run_dignosis"


In [14]:
#adding nodes to the graph.
#nodes is nothing but python function which take state as input and perform some action inside it and rtn the state obect.
graph.add_node(node="sentiment_analysis",action=sentiment_analysis)
graph.add_node(node='run_dignosis',action=run_dignosis)
graph.add_node(node="positive_response",action=positive_response)
graph.add_node(node="negative_response",action=negative_response)

In [15]:
#adding edges to graph.
graph.add_edge(START,"sentiment_analysis")
graph.add_conditional_edges("sentiment_analysis",route_condition) # function that returns the key for the next node

#adding edges
graph.add_edge("run_dignosis","negative_response")

#ending the nodes
graph.add_edge("negative_response",END)
graph.add_edge("positive_response",END)



In [16]:
#compiling the graph to show visual
#graph.compile()

In [17]:
workflow = graph.compile()

#review provided by user.
text = """I bought this smartphone last month and I am really disappointed. 
The battery drains very quickly and it hardly lasts half a day. 
The camera quality is poor, especially in low light, and the pictures come out blurry. 
The phone also hangs frequently while using normal apps. Overall, it feels overpriced and I regret purchasing it."""

#review saving them to state memory.
initial_state = SentimentSchema(review=text)

#invoking the workflow passing initial memory or stATe or sharing to throughout workflow .
response = workflow.invoke(initial_state)
response

{'review': 'I bought this smartphone last month and I am really disappointed. \nThe battery drains very quickly and it hardly lasts half a day. \nThe camera quality is poor, especially in low light, and the pictures come out blurry. \nThe phone also hangs frequently while using normal apps. Overall, it feels overpriced and I regret purchasing it.',
 'sentiment': 'Negative',
 'diagnosis': {'Issue': 'Poor smartphone performance and value, with specific issues in battery life, camera quality, and responsiveness.',
  'Urgency': 'High',
  'Tone': 'disappointed'},
 'response': "I apologize for the disappointing experience with your smartphone, and I'm sorry to hear that it's not meeting your expectations in terms of battery life, camera quality, and performance. I understand your frustration, and I'd like to help - please contact our support team directly so we can troubleshoot the issue and explore possible solutions or alternatives for you."}

In [18]:
# #text = "This phone heats up quickly and the camera quality is very poor compared to what was advertised. Total waste of money."
# text = """ I recently bought this laptop and I am very impressed with its performance.
# The speed is excellent, even with multiple applications running at the same time.
# The display quality is sharp and vibrant, which makes watching movies and working on presentations enjoyable.
# The battery life is also great and lasts me almost a full day of work. 
# Overall, this laptop has exceeded my expectations and I would definitely recommend it to others."""